昨日は、lightGBMに「築年数」「面積」「間取り」「契約期間」「その部屋のある階数」「全体の階数」「所在地」「最寄り駅」「最寄り駅までの所要時間」をいれてみて、過学習に陥った。<br>
本日は正則化を試みる。

In [520]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb

訓練データの読み込み

In [521]:
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange_scores = pd.read_csv('room_arrange_scores.csv')
contract_span = pd.read_csv('contract_span.csv')
floor_scores = pd.read_csv('floor_scores.csv')
Floor_scores = pd.read_csv('capital_floor_scores.csv')
wards = pd.read_csv('wards.csv')
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')

rent = pd.read_csv('rent.csv')

テストデータの読み込み

In [522]:
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange_scores = pd.read_csv('test_room_arrange_scores.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_wards = pd.read_csv('test_wards.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')

訓練データの作成

In [523]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, wards, stations, minits], axis=1)
y_train = rent

In [524]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

テストデータの作成

In [525]:
X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_wards, test_stations, test_minits], axis=1)

lightGBMで正則化を行う。

In [526]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, wards, stations, minits], axis=1)
y_train = rent


X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_wards, test_stations, test_minits], axis=1)

category_lists = ['市区町村','最寄り駅']

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'lambda_l2':0.1
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['市区町村', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
c:

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1089
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 9
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 37666.7	valid_1's rmse: 42918.9
[20]	training's rmse: 23451.3	valid_1's rmse: 31991.2
[30]	training's rmse: 18399.6	valid_1's rmse: 28084.6
[40]	training's rmse: 16219.7	valid_1's rmse: 26579.7
[50]	training's rmse: 15034.1	valid_1's rmse: 25735.5
[60]	training's rmse: 14214	valid_1's rmse: 25165.9
[70]	training's rmse: 13584.7	valid_1's rmse: 24727
[80]	training's rmse: 13071.8	valid_1's rmse: 24491.1
[90]	training's rmse: 12561.9	valid_1's rmse: 24194.3
[100]	training's rmse: 12190.5	valid_1's rmse: 24014
[110]	training's rmse: 11820.8	valid_1's rmse: 23892.7


カテゴリ化がうまくいってない気がする・・・。<br>
カテゴリデータ意外のもののみでlightGBMを走らせる。

In [527]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, minits], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_minits], axis=1)

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'lambda_l2':0.00001
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000715 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 669
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 7
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 40592.2	valid_1's rmse: 45159
[20]	training's rmse: 31519.9	valid_1's rmse: 38615.6
[30]	training's rmse: 28648.7	valid_1's rmse: 36901
[40]	training's rmse: 27327.8	valid_1's rmse: 36181.3
[50]	training's rmse: 26480.2	valid_1's rmse: 35613.8
[60]	training's rmse: 25786.7	valid_1's rmse: 34873
[70]	training's rmse: 25255.8	valid_1's rmse: 34428.8
[80]	training's rmse: 24627.2	valid_1's rmse: 34104.4
[90]	training's rmse: 24113.3	valid_1's rmse: 33639.3
[100]	training's rmse: 23753.8	valid_1's rmse: 

スコアは下がってしまった。カテゴリ化データはあったほうがよさそう。

ほかの特徴量の組み合わせを試してみる。<br>
以下は、最寄り駅と市区町村の情報がかぶっているという観点から「市区町村」を消したもの。

In [528]:
X_train = pd.concat([house_age, area_size, room_arrange_scores, contract_span, floor_scores, Floor_scores, stations, minits], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_room_arrange_scores, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits], axis=1)

category_lists = ['最寄り駅']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'lambda_l2':0.0000001
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\k

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003080 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1065
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 8
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 37643	valid_1's rmse: 42927.7
[20]	training's rmse: 23602.3	valid_1's rmse: 32015.8
[30]	training's rmse: 18486.2	valid_1's rmse: 28132.9
[40]	training's rmse: 16344.2	valid_1's rmse: 26545.6
[50]	training's rmse: 15128.1	valid_1's rmse: 25565.8
[60]	training's rmse: 14319.6	valid_1's rmse: 24945.5
[70]	training's rmse: 13692.4	valid_1's rmse: 24463.2
[80]	training's rmse: 13086.3	valid_1's rmse: 24035
[90]	training's rmse: 12638.2	valid_1's rmse: 23777.2
[100]	training's rmse: 12256.8	valid_1's rmse: 23577.5
[110]	training's rmse: 11908.8	valid_1's rmse: 23341.

今までの中であれば悪くはないが、まだまだ過学習している。

ファイル出力

In [548]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('result.csv',index=False)